# 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거한다. 
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동 평균선, 상단 밴드, 하단 밴드 파생변수 생성
2. 두 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은 ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print) 
3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익율 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익율 변경
    3. 누적 수익율을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익율을 출력(print)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# 1번 함수 생성
# 매개 변수 추가 
def create_band(df, col, start, end):
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)
    # start, end를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    # 데이터를 시작시간부터 종료 시간까지 필터
    df = df.loc[start:end]
    # 결측치와 이상치를 제거 
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns'), [col]]
    # 수정 종가 컬럼을 제외한 데이터프레임 생성
    # df = df[[col]]
    # 이동 평균선 생성
    df['center'] = df.rolling(20).mean()
    # 상단 밴드 생성
    df['ub'] = df['center'] + ( 2 * df[col].rolling(20).std() )
    # 하단 밴드 생성
    df['lb'] = df['center'] - ( 2 * df[col].rolling(20).std() )
    # 결과를 리턴
    return df

In [ ]:
load_data = pd.read_csv("../csv/GM.csv", index_col='Date')

In [ ]:
data = create_band(load_data, 'Close', '20110101', '20150101')

In [ ]:
data2 = create_band(load_data, 'Adj Close')

In [ ]:
data.head(1)

In [ ]:
data2.head(1)

In [ ]:
# 2번째 함수
def add_trade(df):
    # 기준이되는 컬럼이 무엇인가?
    # 기준이 되는 컬럼은 컬럼 중에 첫번째 이기 때문에 df.columns[0]
    col = df.columns[0]
    # trade 파생변수 생성
    df['trade'] = ""
    for i in df.index:
        # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, col] > df.loc[i, 'ub']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 매도
                df.loc[i, 'trade'] = ''
            else:
                df.loc[i, 'trade'] = ''
        # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i, col] < df.loc[i, 'lb']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                # 매수
                df.loc[i, 'trade'] = 'buy' 
        else:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = ''
    return df

In [ ]:
add_trade(data)['trade'].value_counts()

In [ ]:
add_trade(data2)['trade'].value_counts()

In [ ]:
data = add_trade(data)

In [ ]:
# 3번째 함수 생성
def add_rtn(df):
    col = df.columns[0]
    # 수익율 파생변수 생성
    df['return'] = 1
    # 판매한 날의 수익율 대입
    rtn = 1.0
    buy = 0.0
    sell = 0.0

    for i in df.index:
        # 구매가를 출력
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, col]
            print('진입일 :', i, '구매 가격 :', buy)
        # 판매가를 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, col]
            rtn = (sell - buy) / buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매 가격 :', sell, '수익율 :', rtn)

        # 구매가, 판매가를 초기화
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
    
    # 누적 수익율 계산하여 파생변수에 대입
    acc_rtn = 1.0

    for i in df.index:
        rtn = df.loc[i, 'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn

    print('누적 수익율 :', acc_rtn)

    return df


In [ ]:
add_rtn(data)

In [ ]:
import bollinger as bg
import imp
imp.reload(bg)

In [ ]:
df = pd.read_csv("../csv/MSFT.csv", index_col='Date')

In [ ]:
# 1번 함수를 실행
data = bg.create_band(df, 'Adj Close', '20100101', '20190101')

In [ ]:
# 2번 함수 실행
data2 = bg.add_trade(data)

In [ ]:
# 3번 함수를 실행
bg.add_rtn(data2)

In [ ]:

data = pd.read_csv("../csv/AAPL.csv", index_col='Date')

In [ ]:
data.head(5)

In [ ]:
# 클래스 생성
class_a = bg.Invest(data, 'Adj Close')

In [ ]:
class_a.col

In [ ]:
class_a.create_band('20100101', '20150101')

In [ ]:
class_a.df

In [ ]:
class_a.add_trade()

In [ ]:
class_a.df

In [ ]:
class_a.add_rtn()

In [ ]:
class_a.df